In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import io
import boto3
import joblib
import os

ToDo: Should make preprocessing a part of model pipeline to avoid having to preprocess test data at inference time


In [ ]:
bucket_name = "lneg-loka"
data_file_name = "patient_data_raw/patient_data_raw.csv"
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=data_file_name)
csv_string = obj['Body'].read().decode('utf-8')
data = pd.read_csv(io.StringIO(csv_string))


In [ ]:
data

DecisionTree is a suitable first approach for tabular data. Class-conditional feature plots and statistical tests suggest features are not informative for prediction of Chronic Obstructive Pulmonary Disease, so I didn't bother with cross-validation or hyperparameter tuning.

In [ ]:
model_features =['age','bmi','smoker','sex','chronic_obstructive_pulmonary_disease']
df = data[model_features]
df

In [ ]:
target_name = 'chronic_obstructive_pulmonary_disease'
X = df.drop(target_name, axis=1)
y = df[target_name]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [ ]:
model = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=5,
    random_state=42
)


In [ ]:
preprocessor = ColumnTransformer(transformers=[('cat',OneHotEncoder(drop='first'),['sex','smoker'])],remainder='passthrough')

In [ ]:
pipe = Pipeline([("preprocess",preprocessor),("model", model)])
pipe.fit(X_train,y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred)) 


In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
plt.figure(figsize=(20,10))
tree.plot_tree(model, feature_names=X.columns, class_names=True, filled=True)
plt.show()

In [ ]:
model_dir = '.\..\models'
model_name = 'decision_tree_classifier.joblib'


In [ ]:
joblib.dump(pipe,os.path.join(model_dir,model_name))